# Strong Lensing in Twinkles

## Overview

Strong Lensing in Twinkles simulations involves taking existing CATSIM instance catalogs and replacing specific AGN within those catalogs with a Strongly Lensed AGN system. This means adding images of the original AGN and a lens galaxy into the CATSIM instance catalogs. This is done using a piece of code in the Twinkles repository that is known as the sprinkler. This notebook will first discuss the sprinkler and then move onto verifying its outputs in instance catalogs.

## Sprinkler Code Review

The sprinkler code can be found [here](https://github.com/DarkEnergyScienceCollaboration/Twinkles/blob/master/python/desc/twinkles/sprinkler.py). It involves a step inserted into the `final pass` method of the Instance Catalog class that allows manipulation of the array of catalog results to be manipulated before printing out. This step starts with the loading of an OM10 based catalog of strongly lensed AGN systems.

### The OM10-Twinkles Catalog

The catalog we are using started with the set of the 872 rung 4 lenses from the Time Delay Challenge 1 (see discussion in this [issue](https://github.com/DarkEnergyScienceCollaboration/Twinkles/issues/117) on why these were chosen). However, when looking through this catalog we learned that sizes were missing for the foreground lens galaxies and we have chosen to use CATSIM galaxies to add this information to the catalog as well as an SED for each of these foreground galaxies. This process was completed in this [issue](https://github.com/DarkEnergyScienceCollaboration/Twinkles/issues/310). In the process 11 galaxies were unable to be matched adequately to CATSIM galaxies so we end up with a final catalog in Twinkles of 861 galaxies.

### Selecting Sprinkled Systems

The next step after loading the lensing catalog is to go through the instance catalog AGNs one at a time and see if they match a lens system from our OM10-Twinkles catalog. The matching is done by taking the redshift and magnitude of the CATSIM AGN and selecting any available lens systems with a source AGN that has a redshift within 0.1 dex and an LSST magnorm value (magnitude in a simulated filter with one bin at 500nm) within .25 mags.

Once the possible matches to a given instance catalog AGN are found there is a chance that it will be replaced that is set by the density parameter in the sprinkler class. In the current Twinkles run it is set at 1.0 so that 198 lens systems are sprinkled into the Twinkles instance catalogs.

If the AGN is chosen to be sprinkled then we proceed with the sprinkler and if there are multiple matches from the OM10-Twinkles catalog we randomly choose one to use as we proceed. First, we remove any disc and bulge information for that galaxy from the instance catalog leaving only the AGN data and copy this AGN entry creating 2 or 4 entries in the catalog depending upon the number of images the OM10 system we are sprinkling in has in total. Next we use the OM10 data for each image to shift the location of the AGN in each image to the correct location and adjust the magnitude for each entry depending on the OM10 magnification parameter for each AGN image. At this point we also adjust the time delay for each image with OM10 info and give all the images the same source redshift from the OM10 entry. Once the new entries are ready they are appended to the instance catalog with a new instance catalog id tag that gives us the ID of the Bulge galaxy in the instance catalog to which it is associated and the ID number of the OM10-Twinkles catalog system that is used.

The next step is to take the original entry in the instance catalog and add in the information for the foreground lens galaxy. Therefore, in this step the AGN and Disc data for the original galaxy are cleared and the redshift and magnitude for the lens galaxy from the OM10 system is added into the instance catalog. Here we also add in the radius of the lens galaxy and the sed file we want to use (the information added to the original OM10 systems in the process described in the last section). We also add in the OM10 position angles and ellipticities at this point.

Once we have gone through this process for all AGN in the instance catalog then we return to the normal instance catalog processing to write the information out to file.

## Verifying Instance Catalog Output

For the sections below I am using the output from an instance catalog created using Twinkles and removing the SNe entries at the bottom. I then split it up into the bulge+disk galaxy entries and the AGNs available for download [here](https://gist.github.com/jbkalmbach/ca365abf031ee024aeaa22ad571a8f4d) and [here](https://gist.github.com/jbkalmbach/e64d05abf7ea0e5cc5e0fc75116930f8), respectively.

### Image Positions

Our first task is to verify that the image locations in the instance catalog are in the correct places when compared to the entries from the OM10 catalog.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.io import fits

In [57]:
om10_cat = fits.open('../../data/twinkles_lenses_v2.fits')[1].data
galaxy_cat = pd.read_csv('phosim_galaxy_220.txt', delimiter=' ', usecols=(1, 2, 3, 6), names=['id', 'ra', 'dec', 'redshift'])
agn_cat = pd.read_csv('phosim_agn_220.txt', delimiter=' ', usecols=(1, 2, 3, 6), names=['id', 'ra', 'dec', 'redshift'])

Now, we will cut on id numbers larger than a certain size. During the sprinkling process as new id numbers are given to the sprinkled AGN all of their ids are larger than 1e11 while those unsprinkled are not. We will also take the information contained in these larger ids and add columns containing the bulge id number and the om10 catalog id number.

In [58]:
sprinkled_agn = agn_cat.query('id > 1e11').reset_index(drop=True)

In [96]:
twinkles_id = []
im_num = []
lens_gal_id = []
for id_num in sprinkled_agn['id']:
    shift_id = np.right_shift(id_num, 10)
    twinkles_id.append(int(str(shift_id)[-3:])/4)
    im_num.append(int(str(shift_id)[-3:]) % 4)
    lens_gal_id.append(np.left_shift(np.right_shift(id_num-28, 10)/10000, 10) + 26)

In [97]:
sprinkled_agn['twinkles_id'] = twinkles_id
sprinkled_agn['image_num'] = im_num
sprinkled_agn['lens_gal_id'] = lens_gal_id

We now have a dataframe of sprinkled AGN that can be associated with the other two datasets for validation purposes.

In [105]:
sprinkled_agn[:20]

,id,ra,dec,redshift,twinkles_id,image_num,lens_gal_id
0,213934178332,53.052889,-27.703314,0.48,24,0,21393434
1,213934179356,53.052407,-27.702922,0.48,24,1,21393434
2,233595183132,53.083450,-27.707425,0.66,74,0,23359514
3,233595184156,53.083715,-27.707304,0.66,74,1,23359514
4,485837983772,52.785827,-27.304227,0.75,39,0,48583706
5,485837984796,52.785950,-27.304012,0.75,39,1,48583706
6,551311822876,52.969919,-27.422191,0.69,113,0,55131162
7,551311823900,52.969891,-27.421739,0.69,113,1,55131162
8,785592844316,52.900595,-27.281866,0.62,128,0,78559258
9,785592845340,52.900625,-27.281633,0.62,128,1,78559258
